<a href="https://colab.research.google.com/github/dat810/tt2_things/blob/main/present_panic/bomb_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prestige Event Bomb Game

Exploring the following:
- Average scissors needed per board
- Average boards completed for the event

So that you are prepared for the RNG 🤣

## Setting Up
Load libraries

In [ ]:
import random
import numpy as np
import ctypes
import matplotlib.pyplot as plt
import pandas as pd
import multiprocessing as mp
import os
import requests

Load the dataset (as of 7.5.2)

In [ ]:
!git clone https://github.com/rawrzcookie/TT2_CSV.git

Dataset already exists at HolidayEventBombGameLevelInfo.csv


In [ ]:
!cp TT2_CSV/csv/HolidayEventBombGameLevelInfo.csv .

In [ ]:
level_info = pd.read_csv('HolidayEventBombGameLevelInfo.csv')

Define a Board

In [ ]:
%%writefile board.h
#ifndef BOARD_H
#define BOARD_H

typedef struct {
    int rows;
    int columns;
    int bombs;
    int target;
    char difficulty[12];
} Board;

#endif // BOARD_H


In [ ]:
%%writefile board.c
#include <string.h>
#include "board.h"

// Function to determine difficulty from index
void getDifficulty(int diff_idx, char *difficulty) {
    switch (diff_idx) {
        case 0:
            strcpy(difficulty, "Easy");
            break;
        case 1:
            strcpy(difficulty, "Hard");
            break;
        case 2:
            strcpy(difficulty, "Impossible");
            break;
        default:
            strcpy(difficulty, "Unknown");
    }
}


Define a simulator for a board. Return the number of scissors used.



In [ ]:
%%writefile bomb_game_sim.h
#ifndef BOMB_GAME_SIM_H
#define BOMB_GAME_SIM_H

int bomb_game_sim(int boxes, int bombs, int target);

#endif // BOMB_GAME_SIM_H


In [ ]:
%%writefile bomb_game_sim.c
#include <stdlib.h>
#include <time.h>

int bomb_game_sim (int boxes, int bombs, int target) {
    int g = boxes - bombs;
    int total_scissors = 0;
    int streak = 0;
    int *board = (int *)malloc(boxes*sizeof(int));
    int i;

    // Initialize the board: 0 for bombs, 1 for corrects
    for (i=0;i<bombs;i++) {
        board[i] = 0; // bombs
    }
    for (i=bombs;i<boxes;i++) {
        board[i] = 1; // corrects
    }

    srand((unsigned int)time(NULL)+rand());

    while (streak < target) {
        int index = rand() % boxes;
        total_scissors++;

        if (board[index] == 1) { // corrects
            streak++;
            board[index] = board[--boxes]; // Remove corrects and shrink board
        } else { // troll
            streak = 0;
            bombs--;
            boxes = bombs + g; // Reset board
            for (i = 0; i < bombs; i++) board[i] = 0; // bombs
            for (i = bombs; i < boxes; i++) board[i] = 1; // corrects
        }
    }

    free(board);
    return total_scissors;
}

Define a simulator for an event run. Return the number of boards completed.

In [ ]:
%%writefile event_sim.h
#ifndef EVENT_SIM_H
#define EVENT_SIM_H

#include "board.h"

void event_sim(int n, int start, Board *boards, int num_boards, int *results);

#endif // EVENT_SIM_H


In [ ]:
%%writefile event_sim.c
include <stdio.h>
#include <stdlib.h>
#include "event_sim.h"
#include "bomb_game_sim.h"

void event_sim(int n, int start, Board *boards, int num_boards, int *results) {
    for (int j = 0; j < n; j++) {
        int i = 0;
        int scissors = start;

        while (scissors > 0) {
            if (i >= num_boards) {
                printf("Error: No more boards available.\n");
                results[j] = i;
                break;
            }

            int bx = boards[i].rows * boards[i].columns;
            int bm = boards[i].bombs;
            int target = boards[i].target;

            int scissors_used = bomb_game_sim(bx, bm, target);
            scissors -= scissors_used;

            if (scissors > 0) {
                i++;
            }
        }

        results[j] = i;
    }
}


Parse the board information into a dataframe, and create the `Board` objects.

## Average Scissors Needed per Board

In [ ]:
%%writefile main.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "board.h"
#include "event_sim.h"

#define MAX_LINE_LENGTH 1024
#define MAX_BOARDS 100

// Function to parse the CSV file and populate the boards array
int parse_csv(const char *filename, Board *boards) {
    FILE *file = fopen(filename, "r");
    if (file == NULL) {
        perror("Error opening file");
        return -1;
    }

    char line[MAX_LINE_LENGTH];
    int count = 0;

    // Read the header line and ignore it
    if (fgets(line, sizeof(line), file) == NULL) {
        perror("Error reading file");
        fclose(file);
        return -1;
    }

    // Read each subsequent line and parse the columns
    while (fgets(line, sizeof(line), file) && count < MAX_BOARDS) {
        int rows, columns, bombs, successes_needed, chest_icon_index;

        // Tokenize the line and extract required fields
        char *token = strtok(line, ",");
        // First column: Rows
        if (token) rows = atoi(token);

        token = strtok(NULL, ",");
        // Second column: Columns
        if (token) columns = atoi(token);

        token = strtok(NULL, ",");
        // Third column: BombsOnBoard
        if (token) bombs = atoi(token);

        token = strtok(NULL, ",");
        // Fourth column: SuccessesNeededToWin
        if (token) successes_needed = atoi(token);

        token = strtok(NULL, ",");
        // Fifth column: ChestIconIndex
        if (token) chest_icon_index = atoi(token);

        // Fill the board data
        boards[count].rows = rows;
        boards[count].columns = columns;
        boards[count].bombs = bombs;
        boards[count].target = successes_needed;

        // Map chest_icon_index to difficulty
        getDifficulty(chest_icon_index, boards[count].difficulty);

        count++;
    }

    fclose(file);
    return count;
}

int main() {
    // Declare an array to hold board information
    Board boards[MAX_BOARDS];
    int num_boards = parse_csv("HolidayEventBombGameLevelInfo.csv", boards);

    if (num_boards < 0) {
        fprintf(stderr, "Failed to parse CSV file.\n");
        return 1;
    }

    printf("Loaded %d boards from the CSV file.\n", num_boards);

    // Number of simulations and starting scissors
    int n = 10;
    int start = 100;

    // Array to hold results
    int *results = (int *)malloc(n * sizeof(int));
    if (results == NULL) {
        fprintf(stderr, "Memory allocation failed.\n");
        return 1;
    }

    // Run the simulation
    event_sim(n, start, boards, num_boards, results);

    // Print results
    for (int i = 0; i < n; i++) {
        printf("Simulation %d: Cleared %d boards\n", i + 1, results[i]);
    }

    // Free allocated memory
    free(results);

    return 0;
}


## Average Boards Completed in an Event

## Notebook Information
- Notebook content created by dat
- Dataset from rawrzcookie, who also fetched the CSV files here: https://github.com/rawrzcookie/TT2_CSV